# The Variational Hamiltonian Ansatz (VHA) on the Hubbard model 

The Hubbard model is a simplification of correlated electrons. Despite its simplistic structure, the Hubbard model is able to capture some interesting physics, such as the transition of a solid from a conducting to insulating state and some superconducting effects (though I don't yet understand any of this). 

In this notebook, we'll briefly describe the Hubbard model and then try to use the VHA to solve it. 

$\newcommand{\ket}[1]{\lvert #1 \rangle}$
$\newcommand{\bra}[1]{\langle #1 \rvert}$

**Table of contents:** 
1. [The Hubbard model](#Hubbard-model)
    * [A quantum mechanical solid!](#qm-solid)
    * [Defining creation and annihilation operators](#operators)
    * [The Jordan-Wigner transformation](#jw)
    * [Defining the Hubbard Hamiltonian](#hubbard-ham)


**Update 1**: I started by defining my own classes/functions for creation/annihilation operators, the Jordan-Wigner transform, the square lattice for the Hubbard Hamiltonian, the Hubbard Hamiltonian's matrix representation, and the Variational Hamiltonian Ansatz (VHA). As expected, my code and computer's RAM wasn't enough to work with anything more than a 2x2 square lattice (which has 8 qubits, so exists 256-dimensional Hilbert space). What did surprise me was how much more efficient OpenFermion was, because when I tried it when SSHing onto a Google Cloud computer, I was able to quickly solve the ground state of up to a 2x10 square lattice Hubbard Hamiltonian. Most of the functions/classes I defined are still available in the tools/ folder if you want to take a look. 

In [1]:
import numpy as np
import scipy.linalg
import scipy.optimize 

from tools.utils import *
tol = 0.005 # Tolerance for elementwise equality of matrices

<a id="Hubbard-model"></a>
## The Hubbard model 

A question that's been plaguing me since I started studying quantum computing is about its practicality: when do we *need* to use quantum mechanics and what exactly can it tell us about the world that our classical mechanics heuristics can't? 

I don't have a perfect answer (or even a very detailed one) yet, but I tried to use it as a guiding principle for this project. Most introductory quantum computing resources abstract away the quantum mechanics and begin with axioms like *qubits live in a Hilbert space*. This is a great approach, but it's important to *not forget* what you don't understand! I'll try to make it obvious what I'm assuming and what I don't understand throughout this article. 

<a id="qm-solid"></a>
### A quantum mechanical solid! 

An all-too-common story of what papers on practical application of quantum computers is the following: they start with some structure they call "interesting" and then decide, for some unknown reason, to find the ground state energy. *Why* the ground state energy is important is something I don't yet fully understand, but trying to understand that question is a lot easier when we have some more context about the system we're analyzing. (Like most concepts, going "backwards" from abstract theory to specific example can be very enlightening!) 

So, let's start with a problem we want to solve. Then, we'll simplify the problem into a simpler model that we can analyze. 

**TODO: Problem 1.21.a** from Griffiths. It says: "In general, quantum mechanics is relevant when the de Broglie wavelength of the particle in question ($ h / p)$ is greater than the characteristic size of the system ($d$). In thermal equilibrium at (Kelvin) temperature $T$, the average kinetic energy of a particle is 
$$ \frac{p^2}{2m} = \frac{3}{2} k_B T$$
(where $k_B$ is Boltzmann's constant), so the typical de Broglie wavelength is 
$$ \lambda = \frac{h}{\sqrt{3m k_B T}}$$ 
The purpose of this problem is to anticipate which systems will have to be treated quantum mechanically, and which can be safely described classically. 

**Solids.** The lattice spacing in a typical solid is around $d$ = 0.3 nm. Find the temperature below which the free\* *electrons* in a solid are quantum mechanical. Below what temperature are the *nuclei* in a solid quantum mechanical? (Use sodium as a typical case.) *Moral*: The free electrons in a solid are *always* quantum mechanical; the nuclei are almost *never* quantum mechanical. The same goes for liquids (for which the interatomic spacing is roughly the same), with the exception of helium below 4 K. 

\* In a solid the inner electrons are attached to a particular nucleus, and for them the relevant size would be the radius of the atom. But the outermost electrons are not attached, and for them the relevant distance is the lattice spacing. This problem pertains to the *outer* electrons. 

**TODO: Do other electrons not behave quantum mechanically?**

**TODO: Why do we want to know properties of a solid?**

**TODO: Why are electrons important in determining properties of a solid?**

So, we have a solid, and it would be useful to understand how its electrons interact to produce its properties. We know that when we magnify a solid, we find a lattice of atoms. What simplifying assumptions can we make? 
1. Let’s assume the atoms aren’t moving, so we don’t account for vibrating atoms. We approximate this as a lattice of fixed sites. 
2. Let’s assume the atoms have only one electron orbital - Pauli Exclusion Principle says this means we’re restricted to 2 atoms on each site. 
    * Justification: ??
3. All electrons interact via a “screened Coulomb interaction” but let’s only consider interactions between 2 electrons in the same orbital. 
    * Justification: these are the strongest interactions because the electrons are closest. 
    
4. Kinetic energy is the electron moving around the lattice. Let’s assume hopping only occurs between nearest neighbors.
    * Justification: the hopping coefficient can be thought of as the overlap of the wavefunctions, so since wavefunctions die off exponentially, the nearest neighbor hopping coefficients are the largest.  

This is the *Hubbard model*! Here's a visualization of the 2D Hubbard model on a square lattice from [1811.04476](https://arxiv.org/pdf/1811.04476.pdf). 

![](images/2d_hubbard.png)

This is great, but we need some algebraic way to analyze it... 

**TODO:** The following sections need more work, I just copy pasted my initial writing.

<a id="operators"></a>
### Defining creation and annihilation operators 

Since we're describing electrons, the Hubbard Hamiltonian's creation and annihilation operators have the normal fermionic anticommutation relations, namely that for a fermion on site $j,k$ with spin $\sigma, \pi$, 

$$ \large \{ c_{j \sigma}, c^\dagger_{k \pi } \} = \delta_{jk} \delta_{\sigma \pi} \qquad \{ c^\dagger_{j\sigma}, c^\dagger_{k \pi} \}= \{ c_{j\sigma} c_{k\pi} \} = 0  $$

#### Some motivation for these relations 

I don't think there's a very simple explanation of why these relations hold. Even [Feynman admits](https://www.feynmanlectures.caltech.edu/III_04.html) he doesn't have a simple explanation for the mathematical differences for fermions and bosons. Nonetheless, we'll try to extend our knowledge of the quantum harmonic oscillator to fermions. 

Let's start with the latter relation. Fermions are identical particles, so we don't care about how individual particles move and behave, rather choosing to fixate on the properties of the whole. This is why in second quantization, we work in a Fock space. In Fock space, we assign each spin-orbital either $\ket{0}$ if it doesn't contain an electron or $\ket{1}$ if it does. 

From QHO, we know $a^\dagger$ is the creation operator and $a$ is the annihilation operator. We want some relation that makes it clear that $a^2$ and ${a^\dagger}^2$ are both nonsense. Since we're working in a vector space, we can set these both to equal 0, which has the effect of destroying our whole vector space. Notice another way of writing $a^2 = 0$, is $2 a^2 = \{ a, a \} = 0$, which is the anticommutation relation we have. The same logic works for $a^\dagger$. 

Recall from QHO that $a \ket{n} = \sqrt{n} \ket{n-1}$ and $a^\dagger \ket{n} = \sqrt{n + 1} \ket{n+1}$. Let's enforce the rule that $n$ can be only 0 or 1. Now, using the fact that $\sqrt{0} = 0$ and $\sqrt{1} = 1$, we can write $a \ket{n} = n \ket{1-n}$ and $a^\dagger \ket{n} = (1 - n) \ket{1-n}$. 

Now, what is $(a a^\dagger + a^\dagger a) \ket{n}$? It simplifies to $(2n^2 - 2n + 1) \ket{n}$. Plugging in $n = 0$ and $n=1$ results in the same thing: $2n^2 -2n + 1 = 1$. So we can write that $\{ a, a^\dagger \} = 1$. 

<a id="jw"></a>
### The Jordan-Wigner transformation

Now that we have operators to manipulate fermions, we need some way to translate them into operators to manipulate qubits. There are a three main encodings, but the Jordan-Wigner transformation is definitely the most intuitive. 

We let $\mid 0 \rangle$ represent no electron in a spin-orbital, and we let $\mid 1 \rangle$ represent an electron existing in a spin-orbital. This allows us to define the creation operator as $(X-iY)/2$ and the annihilation operator as $(X+iY)/2$. The main problem with this is that the creation and annihilation operators don't anti-commute. To fix this, we prepend the operator with Pauli-$Z$ tensors. Because the Pauli operators anti-commute, this preserves our desired anti-commutation relation. 

<a id="hubbard-ham"></a>
### Defining the Hubbard Hamiltonian 

Okay, now that we have those preliminaries out of the way, what is the Hubbard Hamiltonian? What's the motivation behind it? How is it useful? 

Answer these. 

The Hubbard Hamiltonian: 
$$ \large H = -t \sum_{<j, k>, \sigma} \Big( c^\dagger_{j\sigma} c_{k\sigma} + c^\dagger_{k \sigma} c_{j \sigma} \Big) + U \sum_j n_{j \uparrow} n_{j \downarrow} - \mu \sum_j \Big(n_{j\uparrow} + n_{j\downarrow} \Big) $$

The first term is kinetic energy, a fermion moving from one site to another. The symbol $<j, k>$ implies iterating over sites that are adjacent. 

The second term is interaction energy, additional energy for a doubly-occupied site. 

The third term is chemical potential, which controls the filling. 

**TODO: Is my model getting this final term? It should be fine, because it's still just a quadratic term so we can efficiently find the ground state.**

### 2D Hubbard Hamiltonian on a Square Lattice 


We'll use OpenFermion's `HubbardSquareLattice` class to define our lattice. OpenFermion has a simpler function `fermi_hubbard()` to create a `FermionOperator` to describe our system, but this doesn't give us access to the specific hopping terms in the Hamiltonian (the terms in the first summation above), which we'll need for the Variational Hamiltonian Ansatz. 

The `HubbardSquareLattice` class has a useful method: `site_pair_iter(edge_type)`. We'll use the 'horizontal_neighbor' and 'vertical_neighbor' edge types. But we also need to differentiate between "even" and "odd" horizontal and vertical neighbors: even horizontal neighbors will be horizontal neighbors whose leftmost site has even index, and likewise for horizontal odd, vertical even, and vertical odd neighbors. To get this added specificity, we'll need to examine each item in the iterable generated by `site_pair_iter()`. We need the even and odd terms because then when we Trotterize the Hubbard Hamiltonian, we don't introduce any Trotter error, because the our four sets of tunneling terms (even horizontal, even vertical, odd horizontal, odd vertical) commute with each other. 

Actually, it will be much easier to just subclass `HubbardSquareLattice` and define our own `site_pair_iter()` that allows us to specify 'even' or 'odd'. I'll call this class `DecomposedHubbardSquareLattice`. 

**TODO:** Reorganize and move this to the end, when I decide to fix Trotter error. 

In [2]:
from openfermion.utils import HubbardSquareLattice
#HubbardSquareLattice?

**PROBLEM:** I noticed the source code for how OpenFermion creates the iterator over the neighbors in a Hubbard lattice is that it allows sites at the end to loop around, ie the last site has a tunnelling term with the first site. This is a problem, because 
1. I didn't know this was the case and should look into it more. 
2. I usually write the code for even and odd sites to not loop back to the start. 

I guess this isn't that big of a problem; I can just change my code to loop back to the start. I'll need to do the same checking if `x_dimension` or `y_dimension` are less than 2. 

First, let me check if the Hamiltonians are iso-spectral. I don't think they will be. Wait is this what periodic means?

In [3]:
# HubbardSquareLattice parameters
x_n = 2 
y_n = 2 
n_dofs = 1 # 1 degree of freedom for spin, this might be wrong. Having only one dof means ordered=False. 
periodic = 0 # Not sure what this is, periodic boundary conditions?
spinless = 0 # Has spin

import itertools
class DecomposedHubbardSquareLattice(HubbardSquareLattice):
    @property 
    def edge_types(self):
        # Overriding edge_types property so we can define additional ones 
        return ('onsite', 'neighbor', 'diagonal_neighbor', 'horizontal_neighbor', 'vertical_neighbor', 
                'hor_even_neighbor', 'hor_odd_neighbor', 'ver_even_neighbor', 'ver_odd_neighbor')
        
    def site_pairs_iter(self, edge_type, ordered=True):
        # `ordered` parameter just flips the order: if True, for each a,b -> (a,b), (b,a); if False, for each 
        # a,b -> (a,b), so we only get it once and it doesn't flip order. 
        # We WANT ordered=False, because there's a helper function tunneling_operator(i, j, coeff) that takes in 
        # two sites i,j and does coeff*(i^j j^i). 
        
        # Overriding site_pairs_iter() to add functionality for additional edge_types 
        if edge_type == 'onsite':
            return ((i, i) for i in self.site_indices)
        elif edge_type == 'neighbor':
            return self.neighbors_iter(ordered)
        elif edge_type == 'horizontal_neighbor':
            return self.horizontal_neighbors_iter(ordered)
        elif edge_type == 'vertical_neighbor':
            return self.vertical_neighbors_iter(ordered)
        elif edge_type == 'diagonal_neighbor':
            return self.diagonal_neighbors_iter(ordered)
        # Above was copied from utils._lattice.py so we handle old edge_types correctly. 
        # Below is added functionality for new edge_types. 
        elif edge_type == 'hor_even_neighbor':
            return self.hv_eo_neighbors(lambda x,y: 1-x%2, lambda x,y: (x+1, y), ordered) 
        elif edge_type == 'hor_odd_neighbor':
            return self.hv_eo_neighbors(lambda x,y: x%2, lambda x,y: (x+1, y), ordered)
        elif edge_type == 'ver_even_neighbor':
            return self.hv_eo_neighbors(lambda x,y: 1-y%2, lambda x,y: (x, y+1), ordered)
        elif edge_type == 'ver_odd_neighbor':
            return self.hv_eo_neighbors(lambda x,y: y%2, lambda x,y: (x, y+1), ordered)
        raise ValueError('Edge type {} is not valid.'.format(edge_type))
        
    def hv_eo_neighbors(self, filter_xy, map_next_xy, ordered=True):
        for i in range(self.x_dimension):
            for j in range(self.y_dimension):
                if filter_xy(i, j):
                    # Get indices for next site  
                    k, l = map_next_xy(i, j)
                    # Make sure next site isn't out of bounds 
                    if k >= self.x_dimension or l >= self.y_dimension: continue 
                    
                    site_a = self.to_site_index((i, j))
                    site_b = self.to_site_index((k, l))
                    
                    yield (site_a, site_b)
                    if ordered: yield (site_b, site_a)
        
    # Function OpenFermion uses for 'horizontal_neightbor' edge_type.  
    # PROBLEM: This loops around! Notice the % self.x_dimension. 
    def horizontal_neighbors_iter(self, ordered=True):
        n_horizontal_edges_per_y = (
            self.x_dimension - (self.x_dimension <= 2 or not self.periodic))
        for x in range(n_horizontal_edges_per_y):
            for y in range(self.y_dimension):
                i = self.to_site_index((x, y))
                j = self.to_site_index(((x + 1) % self.x_dimension, y))
                yield (i, j)
                if ordered:
                    yield (j, i)

    # Function OpenFermion uses for 'vertical_neighbor' edge_type
    def vertical_neighbors_iter(self, ordered=True):
        n_vertical_edges_per_x = (self.y_dimension -
                                  (self.y_dimension <= 2 or not self.periodic))
        for y in range(n_vertical_edges_per_x):
            for x in range(self.x_dimension):
                i = self.to_site_index((x, y))
                j = self.to_site_index((x, (y + 1) % self.y_dimension))
                yield (i, j)
                if ordered:
                    yield (j, i)
    
    # I'm changing this function so that it uses my functions, so I can compare the spectrums, since 
    # FermiHubbardModel.hamiltonian() will call this to generate the iterable. 
    def neighbors_iter(self, ordered=True):
        return itertools.chain(
            #self.horizontal_neighbors_iter(ordered),
            #self.vertical_neighbors_iter(ordered)
            
            # itertools.chain('ABC', 'DEF') = Iterable('ABCDEF') 
            self.site_pairs_iter('hor_even_neighbor', ordered), 
            self.site_pairs_iter('hor_odd_neighbor', ordered), 
            self.site_pairs_iter('ver_even_neighbor', ordered), 
            self.site_pairs_iter('ver_odd_neighbor', ordered), 
        )


lattice = DecomposedHubbardSquareLattice(x_n, y_n, n_dofs=n_dofs, periodic=periodic, spinless=spinless)

Now, we'll create a `FermiHubbardModel` instance by passing it our `HubbardSquareLattice` instance defined above. 

To get the `FermionOperator` instance, we need to call `FermiHubbardModel.hamiltonian()`. The [documentation](https://openfermion.readthedocs.io/en/latest/openfermion.html#openfermion.hamiltonians.FermiHubbardModel) isn't that great here, but the [source code](https://github.com/quantumlib/OpenFermion/blob/master/src/openfermion/hamiltonians/_general_hubbard.py) indicates we do indeed get a `FermionOperator` instance which we'll need for calculating ground state, etc. 

We can't just pass an integer for $t$, $U$, or $\mu$ in this class. Instead, we have to specify the *specific* coefficient for each pair and edge type ($t_{ij}^{\textrm{horizontal neighbor}}$). This will be useful later on, I think, because [1506.05135](https://arxiv.org/abs/1506.05135) says we'll need the indices for different values during adiabatic evolution. 

In [4]:
from openfermion.hamiltonians import FermiHubbardModel
#FermiHubbardModel??

In [5]:
from openfermion.utils import SpinPairs
tunneling = [('neighbor', (0, 0), 1.)] # Not sure if this is right
interaction = [('onsite', (0, 0), 2., SpinPairs.DIFF)] # Not sure if this is right
potential = None 
mag_field = 0. 
particle_hole_sym = False # Not sure if this is right

In [6]:
hubbard = FermiHubbardModel(lattice , tunneling_parameters=tunneling, interaction_parameters=interaction, 
                            potential_parameters=potential, magnetic_field=mag_field, 
                            particle_hole_symmetry=particle_hole_sym)

In [7]:
# In tunneling_terms(), there's the following line: 
#                a, aa = param.dofs
#                site_pairs = self.lattice.site_pairs_iter(param.edge_type, a != aa)
# which means we set ordered=False, if we have only one dof. This is wrong behavior right? 
# I think we can correct this by summing both i^j + j^i when we use the iterator.
# UPDATE: Scratch all that, they define and call tunneling_operator(i, j, coeff) to add both terms. 
hubbard.tunneling_parameters[0].dofs

(0, 0)

In [8]:
# Test: Check if the above Hamiltonian is iso-spectral with a Hubbard Hamiltonian with same parameters 
from openfermion.hamiltonians import fermi_hubbard
from openfermion.utils import eigenspectrum
test_hub = fermi_hubbard(x_n, y_n, 1. , 2., chemical_potential=0.0, magnetic_field=mag_field, 
                         periodic=periodic, spinless=spinless, particle_hole_symmetry=particle_hole_sym)
print("Spectrums are the same: ", array_eq(eigenspectrum(test_hub), eigenspectrum(hubbard.hamiltonian()), tol))

Spectrums are the same:  True


Huh. I didn't expect the spectrums to be the same since I removed the loop-back tunneling. 
**Does having same spectrum mean they're the same operator?**

In [9]:
from openfermion.ops import FermionOperator

def tunneling_operator(i, j, coefficient=1.):
    # Copied from hamiltonians/_lattice.py in OpenFermion
    return (FermionOperator(((i, 1), (j, 0)), coefficient) + FermionOperator(
        ((j, 1), (i, 0)), coefficient.conjugate()))
def tunneling_terms_hor_even(hor, even, model):
    # Mostly copied from FermiHubbardMode.tunneling_terms() 
    terms = FermionOperator()
    for param in model.tunneling_parameters:
        a, aa = param.dofs 
        # We don't use param.edge_type because it's 'neighbor' and we need to be more specific
        if hor and even:
            site_pairs = model.lattice.site_pairs_iter('hor_even_neighbor', a != aa)
        elif hor and not even: 
            site_pairs = model.lattice.site_pairs_iter('hor_odd_neighbor', a != aa)
        elif not hor and even: 
            site_pairs = model.lattice.site_pairs_iter('ver_even_neighbor', a != aa)
        elif not hor and not even:
            site_pairs = model.lattice.site_pairs_iter('ver_odd_neighbor', a != aa)

        for r, rr in site_pairs: 
            for spin_index in model.lattice.spin_indices:
                i = model.lattice.to_spin_orbital_index(r, a, spin_index)
                j = model.lattice.to_spin_orbital_index(rr, aa, spin_index)
                terms += tunneling_operator(i, j, -param.coefficient)
    return terms

In [10]:
# All the values in this dictionary are instances of FermionOperator
hamiltonians = {
    'hub': hubbard.hamiltonian(), 
    'non_interacting': hubbard.tunneling_terms(), 
    'hor_even': tunneling_terms_hor_even(True, True, hubbard), 
    'hor_odd': tunneling_terms_hor_even(True, False, hubbard), 
    'ver_even': tunneling_terms_hor_even(False, True, hubbard), 
    'ver_odd': tunneling_terms_hor_even(False, False, hubbard)
}

In [11]:
# Check to make sure summing up the parts gives us the whole of non_interacting term 
print('Sum of horizontal/vertical even/odd terms gives non-interacting term: ', 
      (hamiltonians['hor_even'] + hamiltonians['hor_odd'] + hamiltonians['ver_even'] + 
       hamiltonians['ver_odd'] == hamiltonians['non_interacting']))

Sum of horizontal/vertical even/odd terms gives non-interacting term:  True


## Variational Hamiltonian Ansatz

The VHA is an ansatz inspired by time-evolution of the system. It splits the Hamiltonian into sub-operators and then does time-evolution for those operators: 
$$\large U(\theta) = \prod_{k=1}^n \prod_{\alpha=1}^N \exp \Big( i\theta_{\alpha, k} H_{\alpha} \Big) $$
where $H_\alpha$ are the sub-Hamiltonians and $\theta$ are the parameters being optimized. 

**Question:** Why is this a good ansatz? It seems it only has access to states that are evolutions of the state we start with. Why is that a guarantee that it'll approximate the ground state? 

**Answer:** It's based on adiabatic evolution. I don't understand this but somehow if we start by evolving the ground state of a Hamiltonian (non-interacting part in this case) and then slowly start replacing it with another Hamiltonian (full Hubbard Hamiltonian), we get the corresponding ground state of the new Hamiltonian by magic. 

In [1811.04476](https://arxiv.org/pdf/1811.04476.pdf) they use $N=5$, splitting it as I did above: even and odd horizontal hopping terms, even and odd vertical horizontal terms, and the on-site interaction terms. 

The non-interacting term is quadratic so we can use `openfermioncirq.prepare_gaussian_state` to get its eigenstate. I can basically follow the first part of Tutorial 4 in openfermion-cirq. 

In [12]:
# Compute ground state on GCloud computers

from openfermion import get_sparse_operator, get_ground_state
hub_sparse = get_sparse_operator(hamiltonians['hub'])
# genergy, gstate = get_ground_state(hub_sparse)
# print("Ground state energy: ", genergy)

# "Ground state energy: -3.6272130052966762" 

### Checkout out degeneracies of tunneling term 

In [13]:
# How is get_ground_state() implemented? 
# They use scipy.sparse.linalg.eigsh 

def get_k_lowest_eigenvals_states(sparse_hamiltonian, k):
    # Find n-2 lowest eigenvalues; there's a bug where it doesn't always find k lowest, so instead 
    # I find as many as possible, and then choose k lowest from those 
    n = sparse_hamiltonian.shape[0]
    values, vectors = scipy.sparse.linalg.eigsh(
        sparse_hamiltonian, k=n-2, which='SA') #'SA' means find k smallest algebraic eigenvalues 
    order = np.argsort(values)[:k]
    values = values[order]
    vectors = vectors[:, order]
    return values, vectors

In [14]:
# Lowest eigenvalues of Hubbard term - notice no degeneracies
w_hub, v_hub = get_k_lowest_eigenvals_states(hub_sparse, 1)
w_hub

array([-3.62721301])

In [15]:
from openfermion.utils import is_hermitian
# Lowest eigenvalues of tunneling term 
tun_sparse = get_sparse_operator(hamiltonians['non_interacting'])
is_hermitian(tun_sparse)

True

In [16]:
w_tun, v_tun = get_k_lowest_eigenvals_states(tun_sparse, 16)

In [21]:
print(w)

[-4. -4. -4. -4. -4. -4. -4. -4. -4. -4. -4. -4. -4. -4. -4. -4.]


In [18]:
from openfermion.utils import inner_product
def fidelity(a, b): 
    # Fidelity of pure states 
    inner = inner_product(a, b)
    return np.conjugate(inner) * inner

In [21]:
# Now for these 16 states, calculate fidelity with ground state of Hubbard 
for i in range(len(w_tun)):
    # Make sure we're using smallest eigenvalue
    assert np.abs(w_tun[i] + 4) < tol 
    print(fidelity(v_tun[:,i], v_hub[:,0]))

# We don't have any huge overlaps with ground state... this means adiabatic evolution won't work!

(0.031569908868109134+0j)
(0.04417407280903488+0j)
(0.19856207313862317+0j)
(0.03418121825354631+0j)
(0.046607917333998376+0j)
(0.12622062025255382+0j)
(0.10964243588147862+0j)
(0.03324514709471281+0j)
(0.04404955460026721+0j)
(0.007748407683286344+0j)
(0.07744523099395186+0j)
(0.07668760819445185+0j)
(0.04117657574101663+0j)
(0.04539101349590707+0j)
(0.05627848196905416+0j)
(0.0031673222112651384+0j)


None of the degenerate ground states of the tunneling term is a clear winner for which we should use the initial state for VHA. Instead, let's try doing a perturbation where we add a "tiny part" of the actual Hubbard Hamiltonian to the tunneling Hamiltonian. 

In [60]:
s = 1e-4
perturbed = tun_sparse + s * hub_sparse

w_per,v_per = get_k_lowest_eigenvals_states(perturbed, 16)
for i in range(len(w_per)):
    # Make sure we're using smallest eigenvalue
    assert np.abs(w_per[i] + 4) < tol
    print(fidelity(v_per[:,i], v_hub[:,0]))
    
# WOW, there's a clear winner here v_per[:, 0]

(0.9770315586987669+0j)
(4.709455091746961e-27+0j)
(2.14205428352305e-26+0j)
(1.500557747275894e-26+0j)
(3.158416793736825e-21+0j)
(8.896391917481096e-22+0j)
(1.6740565805997937e-26+0j)
(2.436649781794836e-26+0j)
(2.2785826206914814e-21+0j)
(1.8874169939948695e-22+0j)
(5.987097060050877e-22+0j)
(1.2483860735399268e-28+0j)
(1.164914646070279e-26+0j)
(6.380702664759779e-22+0j)
(3.9306774862790396e-27+0j)
(1.97871291746349e-22+0j)


In [61]:
# We know that v_per[:, 0] is the ground state that has most overlap with our final ground state 
# But v_per[:, 0] is an eigenvector of the perturbed Hamiltonian. 
# Now, we find the tunneling ground state that has most overlap with the perturbed ground state. 

overlaps = []
for i in range(len(w_tun)):
    overlaps.append(fidelity(v_tun[:,i], v_per[:,0]).real)
max_i = np.argmax(overlaps)
max_overlap_state = v_tun[:, max_i]

print("The tunneling ground state with most overlap is v_tun[:, {}], which has an overlap of {}%.".format(
    max_i, 100 * round(overlaps[max_i], 4)))

The tunneling ground state with most overlap is v_tun[:, 2], which has an overlap of 20.32%.


In [62]:
# To pick a specific degenerate ground state, we must specify the orbital_energies. See: 
# https://github.com/quantumlib/OpenFermion/issues/284#issuecomment-375832824
from openfermion import QuadraticHamiltonian,  get_diagonal_coulomb_hamiltonian

# Convert to DiagonalCoulombHamiltonian 
# I think this is because the ansatz uses properties of this?
dc_hub = get_diagonal_coulomb_hamiltonian(hamiltonians['hub'])

orbital_energies, constant = QuadraticHamiltonian(dc_hub.one_body).orbital_energies()
orbital_energies
# Okay so we need to include 0 and 1 because those give us -4, but then we can include any subset of 
# the 4 values that are 0, so we have 2^4 = 16 possible ground states like we expect

array([-2.00000000e+00, -2.00000000e+00, -1.91765100e-16,  1.64346022e-32,
        8.93943564e-17,  5.82889146e-16,  2.00000000e+00,  2.00000000e+00])

In [63]:
# We get all possible combinations of our orbital energies, so we can try them all 

import itertools 
zero_indices = [2,3,4,5]
orbital_energies_combs = []
for r in range(len(zero_indices) + 1):
    for subset in itertools.combinations(zero_indices, r):
        orbital_energies_combs.append([0, 1] + list(subset))
# Checking we have correct subsets
for comb in orbital_energies_combs:
    total = 0
    for i in comb: 
        total += orbital_energies[i]
    assert np.abs(total + 4) < tol

In [64]:
# We generate each ground state, and then compare the state vector to our desired 
# tunneling ground state vector. This way we know which orbital_energies configuration 
# will reproduce it.
from openfermioncirq import prepare_gaussian_state

from cirq import Circuit, final_wavefunction, LineQubit
gstates = []
fidelities = []
for i in range(len(orbital_energies_combs)):
    degen_gstate = final_wavefunction(
        Circuit(
            prepare_gaussian_state(
                # We choose start state to be ground state of tunneling terms
                # But isn't this degenerate?
                LineQubit.range(8), 
                QuadraticHamiltonian(dc_hub.one_body), 
                occupied_orbitals = orbital_energies_combs[i]
            ))
    )
    
    gstates.append(degen_gstate)
    
    fidelities.append(fidelity(degen_gstate, max_overlap_state))

# Pick top 3 fidelities: I reverse np.argsort, then choose first 3
top_three = np.argsort(fidelities)[::-1][:3]
fidelities = np.array(fidelities)[top_three]
gstates = np.array(gstates)[:, top_three]
print("Top 3 fidelities are: ", fidelities)

Top 3 fidelities are:  [0.2032311 +0.j 0.1863635 +0.j 0.15893537+0.j]


Now, we try each of these 3 ground states. One of them is probably the one that adiabatically evolves to the Hubbard ground state. 

In [84]:
# A simple callback function we can pass into the optimizer to tell us what iteration it's on
def gen_callback():
    i = 0 
    def callback(xk):
        nonlocal i
        print("Iteration: ", i)
        i += 1
    return callback

In [85]:
from openfermioncirq import HamiltonianObjective, SwapNetworkTrotterHubbardAnsatz, VariationalStudy
from openfermioncirq.optimization import ScipyOptimizationAlgorithm, OptimizationParams

# Optimizes and saves our VariationalStudy
def run_ansatz(index_orbital_energies_combs):
    steps = 5
    
    obj = HamiltonianObjective(dc_hub) # Define objective function as Hamiltonian averaging
    
    ansatz = SwapNetworkTrotterHubbardAnsatz(x_n, y_n, 1., 2., periodic=False, iterations=steps)
    
    prep_circ = Circuit(
        prepare_gaussian_state(
            ansatz.qubits, 
            QuadraticHamiltonian(dc_hub.one_body), 
            occupied_orbitals=orbital_energies_combs[index_orbital_energies_combs]
        ))
    
    study = VariationalStudy(
        'Hubbard-VHA-{}'.format(index_orbital_energies_combs), 
        ansatz, 
        obj, 
        preparation_circuit=prep_circ
    )
    
    algorithm = ScipyOptimizationAlgorithm(
        kwags={'method': 'Powell', 'callback': gen_callback()}, 
        options={'maxiter': 100}, 
        uses_bounds=False)
    
    optimizaton_params = OptimizationParams(algorithm=algorithm)
    
    # Optimize
    result = study.optimize(optimizaton_params)
    
    study.save()
    return result.optimal_value

In [ ]:
for i in top_three:
    print("For tunneling ground state with index {}, the Hubbard ground state energy is {}".format(
        i, run_ansatz(i)))

In [87]:
# So now we have an error of about 0.12. I suspect this might be the Trotter error, which means Jan-Michael's 
# decomposition will fix it. If it does, then I am justified in working on measurement precision. 
# For now, let's check the overlap with our true ground state

from cirq import resolve_parameters
from openfermioncirq.variational import variational_black_box

def get_optimal_ground_state(study, result): 
    # Modified variational_black_box.UNITARY_SIMULATE.evaluate_noiseless 
    black_box = study._black_box_type(
        study.ansatz, 
        study.objective, 
        study._preparation_circuit, 
        study.initial_state)
    
    circuit = cirq.resolve_parameters(
        black_box.preparation_circuit + black_box.ansatz.circuit, 
        black_box.ansatz.param_resolver(black_box.ansatz.default_initial_params()))
    final_state = circuit.final_wavefunction(
        black_box.initial_state, 
        qubit_order=black_box.ansatz.qubit_permutation(black_box.ansatz.qubits))
    return final_state

In [96]:
# LOAD STUDY WITH BEST VALUE AND CHECK FIDELITY
opt_ground = get_optimal_ground_state(study0, result0)
fidelity(opt_ground, v_hub[:,0])
# 97.7% fidelity is great!

(0.9770259420974481+0j)

How can we implement the above exponentials in a quantum circuit? Hm... 

OF has a `SwapNetworkTrotterHubbard` ansatz. How does it work? 

Well, the SwapNetwork uses only `ISWAP`, `PhasedISWAP`, `CZ` and `Z` gates. What does it do? 

It was proposed in arxiv: 1711.04789. It allows us to simulate a Trotter step in $N$ depth and $N^2 / 2$ two-qubit entangling gates, and lets us prepare arbitrary Slater determinants in $N/2$ depth, all assuming only a linearly connected architecture. 

In [25]:
steps = 5
ansatz = SwapNetworkTrotterHubbardAnsatz(x_n, y_n, 1., 2., periodic=False, iterations=steps)

print('Our ansatz circuit:') 
print(ansatz.circuit.to_text_diagram(transpose=True))

Our ansatz circuit:
0     1             2     3             4     5             6     7
│     │             │     │             │     │             │     │
iSwap─iSwap^(-Th_0) iSwap─iSwap^(-Th_0) iSwap─iSwap^(-Th_0) iSwap─iSwap^(-Th_0)
│     │             │     │             │     │             │     │
×ᶠ────×ᶠ            ×ᶠ────×ᶠ            ×ᶠ────×ᶠ            ×ᶠ────×ᶠ
│     │             │     │             │     │             │     │
│     ×ᶠ────────────×ᶠ    ×ᶠ────────────×ᶠ    ×ᶠ────────────×ᶠ    │
│     │             │     │             │     │             │     │
iSwap─iSwap^(-Tv_0) ×ᶠ────×ᶠ            ×ᶠ────×ᶠ            iSwap─iSwap^(-Tv_0)
│     │             │     │             │     │             │     │
×ᶠ────×ᶠ            │     ×ᶠ────────────×ᶠ    │             ×ᶠ────×ᶠ
│     │             │     │             │     │             │     │
│     @─────────────@^V_0 │             │     @─────────────@^V_0 │
│     │             │     │             │     │             │     │
│ 

In [27]:
# Initial state |v_0> 
n_electrons = x_n * y_n # Half fill
prep_circ = Circuit(
    prepare_gaussian_state(
        # We choose start state to be ground state of tunneling terms
        # But isn't this degenerate?
        ansatz.qubits, 
        QuadraticHamiltonian(dc_hub.one_body), 
        ))

# Create a Hamiltonian variational study 
study = VariationalStudy(
    'Hubbard-VHA', 
    ansatz, 
    obj, 
    preparation_circuit=prep_circ)

print('Created a variational study with {} qubits and {} parameters'.format(
    len(study.ansatz.qubits), study.num_params))

print("The value of the objective with default initial parameters is {}".format(
    study.value_of(ansatz.default_initial_params())))

print("The circuit of the study is:")
print(study.circuit.to_text_diagram(transpose=True))

Created a variational study with 8 qubits and 15 parameters
The value of the objective with default initial parameters is -2.9777673604193895
The circuit of the study is:
0             1                   2                    3                    4                    5                    6                     7
│             │                   │                    │                    │                    │                    │                     │
X             X                   X                    │                    │                    │                    │                     │
│             │                   │                    │                    │                    │                    │                     │
│             │                   PhISwap(0.25)────────PhISwap(0.25)^0.89   │                    │                    │                     │
│             │                   │                    │                    │                    │                    │

In [28]:
# We have 60 parameters. What are they? 
# They're in format (U/T/W/V, p, q, i) where p,q are qubits and i is iteration
# Seems 12 parameters per iteration. Per iteration, 4 interaction terms like we'd expect, 
# and then 8 tunneling terms like we'd expect. 

# How can I make these fewer? YO, SwapNetworkTrotterHubbard has only 3 parameters per iteration!
list(ansatz.params())

[ofc.variational.letter_with_subscripts.LetterWithSubscripts('Th', 0),
 ofc.variational.letter_with_subscripts.LetterWithSubscripts('Tv', 0),
 ofc.variational.letter_with_subscripts.LetterWithSubscripts('V', 0),
 ofc.variational.letter_with_subscripts.LetterWithSubscripts('Th', 1),
 ofc.variational.letter_with_subscripts.LetterWithSubscripts('Tv', 1),
 ofc.variational.letter_with_subscripts.LetterWithSubscripts('V', 1),
 ofc.variational.letter_with_subscripts.LetterWithSubscripts('Th', 2),
 ofc.variational.letter_with_subscripts.LetterWithSubscripts('Tv', 2),
 ofc.variational.letter_with_subscripts.LetterWithSubscripts('V', 2),
 ofc.variational.letter_with_subscripts.LetterWithSubscripts('Th', 3),
 ofc.variational.letter_with_subscripts.LetterWithSubscripts('Tv', 3),
 ofc.variational.letter_with_subscripts.LetterWithSubscripts('V', 3),
 ofc.variational.letter_with_subscripts.LetterWithSubscripts('Th', 4),
 ofc.variational.letter_with_subscripts.LetterWithSubscripts('Tv', 4),
 ofc.varia

In [29]:
# Optimize
algorithm = ScipyOptimizationAlgorithm(
    kwargs={'method': 'Powell'},
    options={'maxiter': 500},
    uses_bounds=False) 
# Try with True? If that doesn't improve, save parameters and see what fidelity is. Then, try adding 
# more iterations. 
optimization_params = OptimizationParams(
    algorithm=algorithm)
result = study.optimize(optimization_params)
print(result.optimal_value)
# GCloud gave -3.0633067159228347

KeyboardInterrupt: 

In [30]:
# study.save() Now I can just load the study with VariationalStudy.load('Hubbard-VHA')
study = VariationalStudy.load('Hubbard-VHA')
result = study.trial_results[0]

In [31]:
opt_ground = get_optimal_ground_state(study, result)

In [69]:
fidelity(opt_ground, gstate)

NameError: name 'gstate' is not defined

In [65]:
# What does this mean?
obj.variance_bound

196.0

In [1]:
from openfermion.utils import commutator 
from openfermion.ops import FermionOperator 
from openfermion.transforms import jordan_wigner 

tunneling_01 = FermionOperator('0^ 1') + FermionOperator('1^ 0')
tunneling_12 = FermionOperator('1^ 2') + FermionOperator('2^ 1')
tunneling_23 = FermionOperator('2^ 3') + FermionOperator('3^ 2')

print('Commutator of 01 and 12 is: ', commutator(tunneling_01, tunneling_12))
print('\nJW matrix of above commutator is: ', commutator(jordan_wigner(tunneling_01), jordan_wigner(tunneling_12)))
print('\nCommutator of 01 and 23 is: ', commutator(tunneling_01, tunneling_23))
print('\nJW matrix of above commutator is: ', commutator(jordan_wigner(tunneling_01), jordan_wigner(tunneling_23)))

Commutator of 01 and 12 is:  1.0 [0^ 1 1^ 2] +
1.0 [0^ 1 2^ 1] +
1.0 [1^ 0 1^ 2] +
1.0 [1^ 0 2^ 1] +
-1.0 [1^ 2 0^ 1] +
-1.0 [1^ 2 1^ 0] +
-1.0 [2^ 1 0^ 1] +
-1.0 [2^ 1 1^ 0]

JW matrix of above commutator is:  0.5j [X0 Z1 Y2] +
-0.5j [Y0 Z1 X2]

Commutator of 01 and 23 is:  1.0 [0^ 1 2^ 3] +
1.0 [0^ 1 3^ 2] +
1.0 [1^ 0 2^ 3] +
1.0 [1^ 0 3^ 2] +
-1.0 [2^ 3 0^ 1] +
-1.0 [2^ 3 1^ 0] +
-1.0 [3^ 2 0^ 1] +
-1.0 [3^ 2 1^ 0]

JW matrix of above commutator is:  0
